# Tweet Intent Classification with Twitter Dataset
<hr>

We will build a intent classification model using GRU model using twitter dataset. The dataset was scrapped using 'Twint'. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 


## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from sklearn.model_selection import KFold

## Load the dataset

In [2]:
df = pd.read_csv('dataset/tweetlabels1000_labeled.xlsx - Sheet2.csv')

In [3]:
print(df.shape)
df

(1003, 4)


,No,Label,Username,Tweet
0,0,none,chyrisalys,wifi watcha pasti indihome
1,1,indirect complaint,woiidal,indihome ada masalah apasih??!!
2,2,remark,ranieaw,sore sore hujan rebahan bareng bocil nonton nu...
3,3,negative remark,daeguv_,"indihome plis untuk tanggal 10,13 jangan kesur..."
4,4,indirect complaint,fiorincha,INDIHOME NGAPASIIII
...,...,...,...,...
998,998,campaign,sangwarior,Bisa sharing bareng sobat indihome gini kan as...
999,999,campaign,sangwarior,Sekarang nyantai dulu bentar sambil scroll sos...
1000,1000,negative remark,untextend,@JefriHandri Sini indihome down dr semalem jam...
1001,1001,campaign,sangwarior,Ini barusan slesei sob... Lumayan buat nyari k...


In [4]:
# drop rows with label = campaign, own tweet, & incomplete
df = df.loc[~df['Label'].isin(['campaign','own tweet','incomplete'])].copy()
df

,No,Label,Username,Tweet
0,0,none,chyrisalys,wifi watcha pasti indihome
1,1,indirect complaint,woiidal,indihome ada masalah apasih??!!
2,2,remark,ranieaw,sore sore hujan rebahan bareng bocil nonton nu...
3,3,negative remark,daeguv_,"indihome plis untuk tanggal 10,13 jangan kesur..."
4,4,indirect complaint,fiorincha,INDIHOME NGAPASIIII
...,...,...,...,...
994,994,inquiry,pecintamochi,"@IndiHomeCare Min, kalo jatuh tempo pembayaran..."
995,995,inquiry,tetehaisyah51,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl..."
997,997,direct complaint,untextend,@IndiHomeCare @fauzindrianto Dari semalem down...
1000,1000,negative remark,untextend,@JefriHandri Sini indihome down dr semalem jam...


In [5]:
# Label Encoding & drop columns
df['label'] = df['Label'].map({'indirect complaint': 0, 'remark': 1, 'negative remark': 2, 'direct compliment' : 3 , 'direct complaint' : 4, 'none' : 5, 'inquiry' : 6})
df.drop(['No', 'Label', 'Username'], axis=1, inplace=True)
df

,Tweet,label
0,wifi watcha pasti indihome,5
1,indihome ada masalah apasih??!!,0
2,sore sore hujan rebahan bareng bocil nonton nu...,1
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2
4,INDIHOME NGAPASIIII,0
...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4
1000,@JefriHandri Sini indihome down dr semalem jam...,2


In [6]:
df.info()
df.groupby( by='label').count()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722 entries, 0 to 1002
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweet   722 non-null    object
 1   label   722 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 16.9+ KB


,Tweet
label,
0,68
1,153
2,128
3,64
4,110
5,123
6,76


In [7]:
# Separate the tweet and label
tweets, labels = list(df.Tweet), list(df.label)

In [8]:
tweets[123]

'@IndiHomeCare Halo min, wifi indihome LOS dari tadi siang! Ini gimna?? Urgent nih gw mau make!!  https://t.co/WRXvsHaAUJ'

## Data Preprocessing
<hr>

for data preprocessing we will do:
- Lower the letter case
- Cleansing (remove ascii, digits, punctuations, extra whitespaces, urls)
- Normalization Indonesian words
- Stemming 'kata berimbuhan'
- Tokenization

The tokenization process will be handled by __Tokenizer__ class in TensorFlow

<b>For padding sequence purpose, one way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [9]:
# to lowercase
df['tweet_lower'] = df['Tweet'].str.lower()
df

,Tweet,label,tweet_lower
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur..."
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii
...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran..."
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl..."
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...


In [10]:
# Regex Manipulation for text cleansing
import re
import string

In [11]:
def text_cleansing(text):
    
       
    # remove non ASCII (emoticon, chinese word, etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    
    # remove digits (using regex) -> subtitute
    text = re.sub('\d+', '', text)
    
    # remove punctuation, reference: https://stackoverflow.com/a/34294398
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # remove whitespace in the beginning and end of sentence
    text = text.strip()
    
    # remove extra whitespace in the middle of sentence (using regex)
    text = re.sub('\s+', ' ', text)
    
    # remove url in tweet (using regex)
    text = re.sub(r"\bhttp\w+", "", text)
    
    return text

In [12]:
# text_cleansing()
df['tweet_clean'] = df['tweet_lower'].apply(lambda x: text_cleansing(x))
df

,Tweet,label,tweet_lower,tweet_clean
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!,indihome ada masalah apasih
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocil nonton nu...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur...",indihome plis untuk tanggal jangan kesurupan
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii,indihome ngapasiiii
...,...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran...",indihomecare min kalo jatuh tempo pembayaran i...
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl...",indihomecare min indihome tv bisa gak berlangg...
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...,indihomecare fauzindrianto dari semalem down g...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...,jefrihandri sini indihome down dr semalem jam ...


In [13]:
# normalization indonesian words
normalized_word = pd.read_csv('dataset/new_kamusalay.csv', header=None)
data_dict = dict(zip(normalized_word[0], normalized_word[1]))
len(data_dict)

15166

In [14]:
def normalize_text(text):
    return ' '.join(data_dict.get(word, word) for word in text.split())

In [15]:
df['tweet_normalized'] = df['tweet_clean'].apply(lambda x: normalize_text(x))
df

,Tweet,label,tweet_lower,tweet_clean,tweet_normalized
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome,wifi watcha pasti indihome,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!,indihome ada masalah apasih,indihome ada masalah apa sih
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocah cilik men...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur...",indihome plis untuk tanggal jangan kesurupan,indihome tolong untuk tanggal jangan kesurupan
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii,indihome ngapasiiii,indihome ngapasiiii
...,...,...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran...",indihomecare min kalo jatuh tempo pembayaran i...,indihomecare min kalau jatuh tempo pembayaran ...
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl...",indihomecare min indihome tv bisa gak berlangg...,indihomecare min indihome televisi bisa tidak ...
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...,indihomecare fauzindrianto dari semalem down g...,indihomecare fauzindrianto dari semalam down t...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...,jefrihandri sini indihome down dr semalem jam ...,jefrihandri sini indihome down dari semalam ja...


In [16]:
# stemming 'kata berimbuhan'
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [17]:
df['tweet_stemmed'] = df['tweet_normalized'].apply(lambda x: stemmer.stem(x))
df

,Tweet,label,tweet_lower,tweet_clean,tweet_normalized,tweet_stemmed
0,wifi watcha pasti indihome,5,wifi watcha pasti indihome,wifi watcha pasti indihome,wifi watcha pasti indihome,wifi watcha pasti indihome
1,indihome ada masalah apasih??!!,0,indihome ada masalah apasih??!!,indihome ada masalah apasih,indihome ada masalah apa sih,indihome ada masalah apa sih
2,sore sore hujan rebahan bareng bocil nonton nu...,1,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocil nonton nu...,sore sore hujan rebahan bareng bocah cilik men...,sore sore hujan rebah bareng bocah cilik tonto...
3,"indihome plis untuk tanggal 10,13 jangan kesur...",2,"indihome plis untuk tanggal 10,13 jangan kesur...",indihome plis untuk tanggal jangan kesurupan,indihome tolong untuk tanggal jangan kesurupan,indihome tolong untuk tanggal jangan surup
4,INDIHOME NGAPASIIII,0,indihome ngapasiiii,indihome ngapasiiii,indihome ngapasiiii,indihome ngapasiiii
...,...,...,...,...,...,...
994,"@IndiHomeCare Min, kalo jatuh tempo pembayaran...",6,"@indihomecare min, kalo jatuh tempo pembayaran...",indihomecare min kalo jatuh tempo pembayaran i...,indihomecare min kalau jatuh tempo pembayaran ...,indihomecare min kalau jatuh tempo bayar indih...
995,"@IndiHomeCare Min, 1 IndiHome TV bisa gak berl...",6,"@indihomecare min, 1 indihome tv bisa gak berl...",indihomecare min indihome tv bisa gak berlangg...,indihomecare min indihome televisi bisa tidak ...,indihomecare min indihome televisi bisa tidak ...
997,@IndiHomeCare @fauzindrianto Dari semalem down...,4,@indihomecare @fauzindrianto dari semalem down...,indihomecare fauzindrianto dari semalem down g...,indihomecare fauzindrianto dari semalam down t...,indihomecare fauzindrianto dari malam down tid...
1000,@JefriHandri Sini indihome down dr semalem jam...,2,@jefrihandri sini indihome down dr semalem jam...,jefrihandri sini indihome down dr semalem jam ...,jefrihandri sini indihome down dari semalam ja...,jefrihandri sini indihome down dari malam jam ...


In [18]:
# Separate the tweet and label
tweets, labels = list(df.tweet_stemmed), list(df.label)

In [19]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", tweets[123])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(tweets)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(tweets)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[123])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[123])

Example of sentence:  indihomecare halo min wifi indihome los dari tadi siang ini bagaimana urgent nih gue mau make
Into a sequence of int: [5, 99, 35, 33, 2, 153, 18, 193, 196, 8, 62, 1091, 37, 34, 22, 366]
Into a padded sequence: [   5   99   35   33    2  153   18  193  196    8   62 1091   37   34
   22  366    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [21]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
indihome 2
tidak 3
di 4
indihomecare 5
yang 6
sudah 7
ini 8
ya 9
pakai 10
2162



## Bidirectional GRU Model with Random Embedding

In [52]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=7, activation='sigmoid')
    ])
    
    model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [53]:
# model_0 = define_model( input_dim=1000, max_length=100)
# model_0.summary()

In [54]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the model

In [57]:
from keras.utils.np_utils import to_categorical

# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(n_splits=10, shuffle=False, random_state=None)

# Separate the tweet and label
tweets, labels = list(df.tweet_stemmed), list(df.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(tweets):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(tweets[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(tweets[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # one-hot
    onehot_train_y = to_categorical(train_y)
    onehot_test_y = to_categorical(test_y)

    # encode data using Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, onehot_train_y, batch_size=32, epochs=30, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, onehot_test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, onehot_test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/30
21/21 [==============================] - 20s 444ms/step - loss: 1.9041 - accuracy: 0.2727 - val_loss: 1.9478 - val_accuracy: 0.2055
Epoch 2/30
21/21 [==============================] - 6s 269ms/step - loss: 1.7406 - accuracy: 0.3713 - val_loss: 1.9767 - val_accuracy: 0.2192
Epoch 3/30
21/21 [==============================] - 6s 303ms/step - loss: 1.4783 - accuracy: 0.5285 - val_loss: 1.8546 - val_accuracy: 0.2877
Epoch 4/30
21/21 [==============================] - 6s 302ms/step - loss: 1.0371 - accuracy: 0.6764 - val_loss: 1.8903 - val_accuracy: 0.3014
Epoch 5/30
21/21 [==============================] - 7s 347ms/step - loss: 0.6390 - accuracy: 0.8197 - val_loss: 2.0631 - val_accuracy: 0.2603
Epoch 6/30
21/21 [==============================] - 6s 291ms/step - loss: 0.3813 - accuracy: 0.9045 - val_loss: 2.1974 - val_accuracy: 0.3014
Epoch 7/30
21/21 [==============================] - 6s 284ms/step - loss: 0.2229 - accuracy: 0.9538 - val_loss: 2.3668 - val_accuracy:

Epoch 9/30
21/21 [==============================] - 6s 302ms/step - loss: 0.1175 - accuracy: 0.9708 - val_loss: 1.9759 - val_accuracy: 0.3750
Epoch 10/30
21/21 [==============================] - 6s 269ms/step - loss: 0.0972 - accuracy: 0.9754 - val_loss: 2.4309 - val_accuracy: 0.3056
Epoch 11/30
21/21 [==============================] - 6s 268ms/step - loss: 0.0753 - accuracy: 0.9800 - val_loss: 2.4261 - val_accuracy: 0.3611
Epoch 12/30
21/21 [==============================] - 6s 268ms/step - loss: 0.0835 - accuracy: 0.9754 - val_loss: 2.5511 - val_accuracy: 0.3194
Epoch 13/30
21/21 [==============================] - 6s 295ms/step - loss: 0.0592 - accuracy: 0.9831 - val_loss: 2.5803 - val_accuracy: 0.3611
Epoch 00013: early stopping
Test Accuracy: 43.05555522441864
Training 5: 
Epoch 1/30
21/21 [==============================] - 20s 461ms/step - loss: 1.9121 - accuracy: 0.2677 - val_loss: 1.8672 - val_accuracy: 0.4167
Epoch 2/30
21/21 [==============================] - 6s 281ms/step - l

21/21 [==============================] - 8s 358ms/step - loss: 0.6511 - accuracy: 0.8369 - val_loss: 1.4200 - val_accuracy: 0.4583
Epoch 6/30
21/21 [==============================] - 9s 411ms/step - loss: 0.3733 - accuracy: 0.9200 - val_loss: 1.3672 - val_accuracy: 0.5139
Epoch 7/30
21/21 [==============================] - 8s 402ms/step - loss: 0.2427 - accuracy: 0.9369 - val_loss: 1.7321 - val_accuracy: 0.4306
Epoch 8/30
21/21 [==============================] - 8s 401ms/step - loss: 0.1712 - accuracy: 0.9646 - val_loss: 1.5519 - val_accuracy: 0.4583
Epoch 9/30
21/21 [==============================] - 8s 401ms/step - loss: 0.1107 - accuracy: 0.9692 - val_loss: 1.7300 - val_accuracy: 0.5000
Epoch 10/30
21/21 [==============================] - 9s 406ms/step - loss: 0.0903 - accuracy: 0.9769 - val_loss: 1.7352 - val_accuracy: 0.4444
Epoch 11/30
21/21 [==============================] - 9s 414ms/step - loss: 0.0760 - accuracy: 0.9862 - val_loss: 1.8240 - val_accuracy: 0.4861
Epoch 12/30
21/

C:\Users\930341\AppData\Local\Temp\ipykernel_2136\2628855945.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  record = record.append(temp, ignore_index=True)


In [58]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,30.136988,42.465752,50.0,43.055555,47.222221,52.777779,54.166669,51.38889,41.666666,47.222221,46.010274


report = record
report = report.to_excel('GRU.xlsx', sheet_name='random')

## Predict new tweet

In [59]:
# define the preprocessing pipeline
def text_preprocessing(text):
    # case folding
    text = text.lower()
    
    # text cleansing
    text = text_cleansing(text)
    
    # text normalization
    text = normalize_text(text)
    
    # stemming
    text = stemmer.stem(text)
    
    return text

In [84]:
from keras.utils.np_utils import to_categorical

# unseen tweet
new_tweet = "gimana cara langganan @indihome?"

# preprocess
text_cleaned = text_preprocessing(new_tweet)
text_sequence = tokenizer.texts_to_sequences([text_cleaned])

max_len= max_length(training_sequences)

seq_padded = pad_sequences(text_sequence, maxlen=max_len, padding=padding_type, truncating=trunc_type)

# prediction
pred_proba = model.predict(seq_padded)

np.argmax(pred_proba, axis=1)


array([6], dtype=int64)